#### About Datasets:
    1. id : unique id for a news article
    2. title: the title of a news article
    3. author: author of the news article
    4. text: the text of the article, could be incomplete
    5. label: a label that marks whether the news article is real or fake.
        1 : Fake News
        2 : Real News

In [5]:
# Import the dependencies
import numpy as np
import pandas as pd
import re # regular expression??
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [7]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\amanu\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [10]:
print(len(stopwords.words('English')))
print(stopwords.words('English'))

179
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than

In [ ]:
# Data Pre-processing

In [19]:
# read data
news_df = pd.read_csv(r'Dataset/train.csv')

In [25]:
news_df.head(5)

id                                              title              author  \
0   0  House Dem Aide: We Didn’t Even See Comey’s Let...       Darrell Lucus   
1   1  FLYNN: Hillary Clinton, Big Woman on Campus - ...     Daniel J. Flynn   
2   2                  Why the Truth Might Get You Fired  Consortiumnews.com   
3   3  15 Civilians Killed In Single US Airstrike Hav...     Jessica Purkiss   
4   4  Iranian woman jailed for fictional unpublished...      Howard Portnoy   

                                                text  label  
0  House Dem Aide: We Didn’t Even See Comey’s Let...      1  
1  Ever get the feeling your life circles the rou...      0  
2  Why the Truth Might Get You Fired October 29, ...      1  
3  Videos 15 Civilians Killed In Single US Airstr...      1  
4  Print \nAn Iranian woman has been sentenced to...      1

In [21]:
news_df.shape

(20800, 5)

In [24]:
news_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [26]:
news_df.isnull().sum() # show missing values in rows
# hum isme ye kaise check karenge ki title aur author mai 2no mai Null kisi kis mai hai
# 

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [27]:
# replacing the null value in the dataset with empty string.(ou you may drop)
news_df = news_df.fillna('')

In [28]:
news_df.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [29]:
# we use only 2 features for model building.(author & title)
# merging the author name and news title.
news_df['content'] = news_df['author']+' '+news_df['title']

In [33]:
news_df['content']

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20795    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797    Michael J. de la Merced and Rachel Abrams Macy...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799              David Swanson What Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object

In [34]:
# seperating the data & label
X = news_df.drop(columns=['label'],axis=1)
Y = news_df['label']

In [35]:
print(X.shape , Y.shape)

(20800, 5) (20800,)


In [ ]:
# Stemming Procedure [ use PorterStemmer]
#     It is the process of reducing a word to its Root Word
#      e.g - love,loving,loved,lover ----> love
#            actor,actress,acting -------> act

In [43]:
port_stem = PorterStemmer()

In [47]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]', ' ', content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [45]:
'''
re.sub krne se [ re.sub('[^a-zA-Z]', ' ', content) ]

 input ----> Jason Chaffetz (@jasoninthehouse) October 28, 2016 \nOf course
 output ---> Jason Chaffetz   jasoninthehouse  October            Of course
 1st argument ---> aaise filter ho jaayega (mtlb ki ye sirf alphabets ko hi filter krega aur sbka remove kr dega)
 2nd argument ---> saare non-alphabets ko space se replace kr dega
 3rd argument ---> column jiise content mile( mtlb text row )
 
'''

"\nre.sub krne se [ re.sub('[^a-zA-Z]', ' ', content) ]\n\n input ----> Jason Chaffetz (@jasoninthehouse) October 28, 2016 \nOf course\n output ---> Jason Chaffetz   jasoninthehouse  October            Of course\n 1st argument ---> aaise filter ho jaayega (mtlb ki ye sirf alphabets ko hi filter krega aur sbka remove kr dega)\n 2nd argument ---> saare non-alphabets ko space se replace kr dega\n 3rd argument ---> column jiise content mile( mtlb text row )\n \n"

In [48]:
# apply this function in content column
news_df['content'] = news_df['content'].apply(stemming)

In [49]:
print(news_df['content'])

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exercis ...
20799                            david swanson keep f aliv
Name: content, Length: 20800, dtype: object


In [53]:
# seperating 'again' the data & label
X = news_df['content'].values
Y = news_df['label']

In [54]:
print(len(news_df['content'].values))
news_df['content'].values

20800


array(['darrel lucu hous dem aid even see comey letter jason chaffetz tweet',
       'daniel j flynn flynn hillari clinton big woman campu breitbart',
       'consortiumnew com truth might get fire', ...,
       'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time',
       'alex ansari nato russia hold parallel exercis balkan',
       'david swanson keep f aliv'], dtype=object)

In [55]:
print(X.shape)

(20800,)


In [56]:
print(Y.shape)

(20800,)


In [59]:
# converting the textual data to numerical data
vectorizer = TfidfVectorizer()
vectorizer.fit(X)
X = vectorizer.transform(X)

In [62]:
print(X.shape)

(20800, 17128)


In [63]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.2, stratify=Y, random_state=2)

In [64]:
print(X_train.shape, X_test.shape)

(16640, 17128) (4160, 17128)


In [65]:
print(Y_train.shape, Y_test.shape)

(16640,) (4160,)


In [ ]:
# training the model : Logistic Regression

In [68]:
model = LogisticRegression()

In [69]:
model.fit(X_train, Y_train)

LogisticRegression()

In [ ]:
# Evaluation

In [70]:
# accuracy_score on the training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)
print(training_data_accuracy)

0.9865985576923076


In [71]:
# accuracy_score on the test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)
print(test_data_accuracy)

0.9790865384615385


In [ ]:
# Making Predictive System

In [72]:
X_new = X_test[0]

prediction = model.predict(X_new)
print(prediction)

if prediction[0]==0:
    print('Beware - News is Fake.')
else:
    print('Legit News.')

[1]
Legit News.


In [73]:
Y_test[0]

1

In [ ]:
# The End